# 1. Overview

# 2. Data understanding

## 2.1 Data description

## 2.2 Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 2.3 Functions

In [2]:
# Define the function to clean and preprocess tweets
def clean_text(text):
    if not isinstance(text, str):
        return text
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove strange characters and punctuation
    strange_chars = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~“!#Ûª'
    text = text.translate(str.maketrans(strange_chars, ' ' * len(strange_chars)))
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text


In [3]:
def preprocess_product_mention(df):
    df['product_mention'] = df['product_mention'].astype(str)
    return df

# 3. Code

## 3.1 Import the database

In [4]:
# Load and prepare data
df = pd.read_csv('df.csv')
df.head()

,tweet_text,emotion_type,product_mention
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion,Apple
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion,Apple
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion,Apple
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion,Apple
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion,Google


In [5]:
# Apply cleaning
df['cleaned_text'] = df['tweet_text'].apply(clean_text)

# Prepare features and labels
texts = df['cleaned_text']
product_mentions = df['product_mention']
labels = df['emotion_type']

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Define preprocessing for text and product mention
text_preprocessor = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, max_df=0.7, min_df=5))
])

product_mention_preprocessor = Pipeline([
    ('preprocessor', FunctionTransformer(preprocess_product_mention, validate=False)),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# Create Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cleaned_text', text_preprocessor, 'cleaned_text'),
        ('product_mention', product_mention_preprocessor, 'product_mention')
    ]
)

# Define models
def build_lstm_model(epochs=10):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_bidirectional_lstm_model(epochs=10):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))
    model.add(Bidirectional(LSTM(128)))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_cnn_lstm_model(epochs=10):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_deep_nn_model(epochs=10):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# XGBoost model
def build_xgboost_model():
    return XGBClassifier(eval_metric='mlogloss')

# Create a pipeline with preprocessing and model
def create_pipeline(model_func, **kwargs):
    return Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model_func(**kwargs))
    ])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df[['cleaned_text', 'product_mention']], labels_categorical, test_size=0.2, random_state=42)

# Define parameter grids for tuning
param_grids = {
    'Logistic Regression': {
        'classifier__C': [0.1, 1, 10]
    },
    'Random Forest': {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [10, 20]
    },
    'XGBoost': {
        'classifier__n_estimators': [100, 200],
        'classifier__learning_rate': [0.01, 0.1]
    },
    'LSTM': {'classifier__epochs': [5, 10, 15]},
    'Bidirectional LSTM': {'classifier__epochs': [5, 10, 15]},
    'CNN + LSTM': {'classifier__epochs': [5, 10, 15]},
    'Deep NN': {'classifier__epochs': [5, 10, 15]}
}

In [6]:
# Define models
model_funcs = {
    'Logistic Regression': lambda: LogisticRegression(max_iter=1000),
    'Random Forest': lambda: RandomForestClassifier(),
    'XGBoost': build_xgboost_model,
    'LSTM': build_lstm_model,
    'Bidirectional LSTM': build_bidirectional_lstm_model,
    'CNN + LSTM': build_cnn_lstm_model,
    'Deep NN': build_deep_nn_model
}

# Create and tune models
results = {}

for name, model_func in model_funcs.items():
    print(f"\nTuning model: {name}")
    if name in ['LSTM', 'Bidirectional LSTM', 'CNN + LSTM', 'Deep NN']:
        pipeline = create_pipeline(model_func, epochs=10)
    else:
        pipeline = create_pipeline(model_func)
    
    grid_search = GridSearchCV(pipeline, param_grids[name], cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1) if name not in ['XGBoost'] else y_pred
    y_true_classes = np.argmax(y_test, axis=1)
    accuracy = grid_search.best_score_
    results[name] = {
        'best_params': grid_search.best_params_,
        'accuracy': accuracy,
        'y_pred': y_pred,
        'classification_report': classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_),
        'confusion_matrix': confusion_matrix(y_true_classes, y_pred_classes)
    }
    print(f"\n{name} Best Parameters: {grid_search.best_params_}")
    print(f"{name} Best Accuracy: {accuracy:.4f}")

    # Print Classification Report
    print(f"\n{name} Classification Report:")
    print(results[name]['classification_report'])

    # Print Confusion Matrix
    print(f"\n{name} Confusion Matrix:")
    cm = results[name]['confusion_matrix']
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'{name} Confusion Matrix')
    plt.show()

# Compare results
best_model_name = max(results, key=lambda x: results[x]['accuracy'])
print(f"\nBest Model: {best_model_name} with Accuracy: {results[best_model_name]['accuracy']:.4f}")


Tuning model: Logistic Regression


ValueError: 
All the 9 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Usuario\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexes\base.py", line 2895, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 98, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index_class_helper.pxi", line 93, in pandas._libs.index.Int64Engine._check_type
KeyError: 'product_mention'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Usuario\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_